# Income prediction on census data

# Objective: 
To predict whether income exceeds 50K/yr based on census data

Dataset: Adult Data Set

https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data


Variable description:
    
age: continuous

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

class: >50K, <=50K

In [1]:
# Pandas and Numpy libraries
import pandas as pd
import numpy as np

In [2]:
# For preprocessing the data
#from sklearn.preprocessing import Imputer
from sklearn import preprocessing

In [3]:
# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split

In [4]:
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB

In [5]:
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score

In [6]:
census_df = pd.read_csv('adult.data', header = None, delimiter=' *, *', engine='python')

Load the dataset. Observe that this file has .data extention

For importing the census data, we are using pandas read_csv() method. This method is a very simple and fast method for importing 
data.

We are passing four parameters. The ‘adult.data’ parameter is the file name. The header parameter is for giving details to pandas
that whether the first row of data consists of headers or not. In our dataset, there is no header. So, we are passing None.

The delimiter parameter is for giving the information the delimiter that is separating the data. Here, we are using ‘ , ’ 
delimiter. This delimiter is to show delete the spaces before and after the data values. This is very helpful when there is 
inconsistency in spaces used with data values.

In [8]:
# Print columns in the adult data set
census_df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [9]:
# Adding headers to the dataframe 
census_df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
                    'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

In [10]:
# Number of records(rows) in the dataframe
len(census_df)

32561

In [11]:
# Handling missing data
# Test whether there is any null value in our dataset or not. We can do this using isnull() method.
census_df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

The above output shows that there is no “null” value in our dataset.

Let’s try to test whether any categorical attribute contains a “?” in it or not. At times there exists “?” or ” ” in place of 
missing values. Using the below code snippet we are going to test whether adult_df data frame consists of categorical variables 
with values as “?”.

In [12]:
for value in ['workclass','education','marital_status','occupation','relationship','race','sex','native_country','income']:
    print(value,":", sum(census_df[value] == '?'))

workclass : 1836
education : 0
marital_status : 0
occupation : 1843
relationship : 0
race : 0
sex : 0
native_country : 583
income : 0


The output of the above code snippet shows that there are 1836 missing values in workclass attribute. 1843 missing values in 
occupation attribute and 583 values in native_country attribute.

# Data preprocessing

For preprocessing, we are going to make a duplicate copy of our original dataframe.We are duplicating adult_df to adult_df_rev 
dataframe. Observe that we have used deep copy while copying. Why?

In [14]:
## Deep copy of adult_df
census_df_rev = census_df.copy(deep=True)

Before doing missing values handling task, we need some summary statistics of our dataframe. For this, we can use describe() 
method. It can be used to generate various summary statistics, excluding NaN values.

In [15]:
census_df_rev.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


We are passing an “include” parameter with value as “all”, this is used to specify that. we want summary statistics of all the 
attributes.

In [16]:
census_df_rev.describe(include= 'all')

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22696,NaN,10501,NaN,14976,4140,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


# Data imputation 


In [17]:
for value in ['workclass','education','marital_status','occupation','relationship','race','sex','native_country','income']:
    replaceValue = census_df_rev.describe(include='all')[value][2]
    census_df_rev[value][census_df_rev[value]=='?'] = replaceValue

/Users/vverdhan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# Hot Encoding 
le = preprocessing.LabelEncoder()
workclass_category = le.fit_transform(census_df.workclass)
education_category = le.fit_transform(census_df.education)
marital_category   = le.fit_transform(census_df.marital_status)
occupation_category = le.fit_transform(census_df.occupation)
relationship_category = le.fit_transform(census_df.relationship)
race_category = le.fit_transform(census_df.race)
sex_category = le.fit_transform(census_df.sex)
native_country_category = le.fit_transform(census_df.native_country)

In [20]:
#initialize the encoded categorical columns
census_df_rev['workclass_category'] = workclass_category
census_df_rev['education_category'] = education_category
census_df_rev['marital_category'] = marital_category
census_df_rev['occupation_category'] = occupation_category
census_df_rev['relationship_category'] = relationship_category
census_df_rev['race_category'] = race_category
census_df_rev['sex_category'] = sex_category
census_df_rev['native_country_category'] = native_country_category

In [21]:
census_df_rev.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,...,native_country,income,workclass_category,education_category,marital_category,occupation_category,relationship_category,race_category,sex_category,native_country_category
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,United-States,<=50K,7,9,4,1,1,4,1,39
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,United-States,<=50K,6,9,2,4,0,4,1,39
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,United-States,<=50K,4,11,0,6,1,4,1,39
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,United-States,<=50K,4,1,2,6,0,2,1,39
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,Cuba,<=50K,4,9,2,10,5,2,0,5


In [22]:
#drop the old categorical columns from dataframe
dummy_fields = ['workclass','education','marital_status','occupation','relationship','race', 'sex', 'native_country']
census_df_rev = census_df_rev.drop(dummy_fields, axis = 1)

In [23]:
census_df_rev = census_df_rev.reindex_axis(['age', 'workclass_category', 'fnlwgt', 'education_category',
                                    'education_num', 'marital_category', 'occupation_category',
                                    'relationship_category', 'race_category', 'sex_category', 'capital_gain',
                                    'capital_loss', 'hours_per_week', 'native_country_category', 
                                    'income'], axis= 1)
census_df_rev.head(5)

AttributeError: 'DataFrame' object has no attribute 'reindex_axis'

In [24]:
census_df_rev = census_df_rev.reindex(['age', 'workclass_category', 'fnlwgt', 'education_category',
                                    'education_num', 'marital_category', 'occupation_category',
                                    'relationship_category', 'race_category', 'sex_category', 'capital_gain',
                                    'capital_loss', 'hours_per_week', 'native_country_category', 
                                    'income'], axis= 1)
census_df_rev.head(5)

,age,workclass_category,fnlwgt,education_category,education_num,marital_category,occupation_category,relationship_category,race_category,sex_category,capital_gain,capital_loss,hours_per_week,native_country_category,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,<=50K
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,<=50K
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,<=50K
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,<=50K
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,<=50K


# Data Slicing

In [25]:
X = census_df_rev.values[:,:14]
Y = census_df_rev.values[:,14] 

In [26]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 5)

Implement Gaussian Naive Bayes

In [27]:
clf = GaussianNB()
clf.fit(X_train, Y_train)

GaussianNB()

In [28]:
Y_pred = clf.predict(X_test)

In [29]:
accuracy_score(Y_test, Y_pred, normalize = True)

0.7903205994349588